# Imports

In [1]:
import sys
import json
import logging
import joblib
import numpy as np
import matplotlib.pyplot as plt

import optuna

from sklearn.model_selection import cross_val_score, ShuffleSplit
from sklearn.linear_model import LinearRegression, SGDRegressor, ElasticNet, BayesianRidge, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.pipeline import Pipeline

from typing import Union

from bb_energy_prediction import data_utils, sklearn_regressors

c:\Users\thodo\miniconda3\envs\energy-prediction\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data

In [2]:
data_df = data_utils.get_data_df(data_path=f"../energy_data/data.pkl")
if "bb_embeddings" in data_df.columns:
    data_df = data_df.drop(columns="bb_embeddings")
data_df = data_df.sample(frac=1)
data_df.head(10)

,bb,energy,program_name
145775,"[movq %rcx, mov %rcx %rdx, mov %r11 %rsi, mov ...",2.884094,function_pointer
34647,"[mov %rcx, nopw %ax, mov %rdi %rax, mov %rdi %...",0.000583,find_biggest
510665,"[movq %rdx (%rsi), jmp]",0.259610,variable_name
5424,[jb 0x7],0.012634,bubblesort
440869,"[lea %rdi, movq %rdi, movq %rdi, movq %rax, jmp]",0.519218,variable_name
455246,"[sub $0x1 %rax, cmpb $0xa (%rax), jz 0x111]",0.151438,variable_name
317234,"[cmp %rax %r12, jb]",0.014741,simple_sort
138583,"[mov %rcx %rax, jmp]",0.508958,function_pointer
260558,"[cmpq $0x0, jz 0x4a]",0.221693,polinominal
133856,"[add $0x8 %rsp, popq %rbx, popq %rbp, popq %r1...",0.000079,function_pointer


In [3]:
program_names = data_df.program_name.value_counts().index[4:]
#test_programs = np.random.choice(program_names, 3, replace=False)
test_programs = ["faa", "quicksort", "declare"]
train_df = data_df[~data_df.program_name.isin(test_programs)]
test_df = data_df[data_df.program_name.isin(test_programs)]

print(f"Test programs: {test_programs}")
print(f"Test data size: {len(test_df)}")

Test programs: ['faa', 'quicksort', 'declare']
Test data size: 28575


In [4]:
X_train = np.array([" ".join(bb) for bb in train_df.bb.tolist()])
y_train = train_df.energy.values

X_test = np.array([" ".join(bb) for bb in test_df.bb.tolist()])
y_test = test_df.energy.values

In [5]:
cnt_vect = CountVectorizer()
cnt_vect.fit_transform(X_train)
vocab_len = len(cnt_vect.get_feature_names_out())
print(f"Vocab length: {vocab_len}")

Vocab length: 1258


# Regressors

In [6]:
exp_dir = "/Users/thodo/Documents/sxoli/diplomatiki/basic-block-energy-prediction/model_checkpoints/regressors"

In [7]:
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
storage_name = "sqlite:////Users/thodo/Documents/sxoli/diplomatiki/optuna-studies/sklearn-regressors.db"

## Linear Regression

Simple least squares regression.
No need for extra tuning.

In [8]:
study = optuna.create_study(study_name="linear-regression", storage=storage_name, load_if_exists=True, direction="minimize")
study.set_user_attr("Loss", "RMSE")

[I 2023-02-13 10:08:23,477] Using an existing study with name 'linear-regression' instead of creating a new one.


Using an existing study with name 'linear-regression' instead of creating a new one.


In [9]:
def objective(trial):

    tfidf = trial.suggest_categorical("tfidf", [True, False])
    normalization = trial.suggest_categorical("normalization", [True, False])
    scaling = trial.suggest_categorical("scaling", [True, False])

    regressor = LinearRegression()
    score = sklearn_regressors.evaluate_regressor(
        regressor=regressor,
        X=X_train,
        y=y_train,
        tfidf=tfidf,
        normalization=normalization, 
        scaling=scaling
    )

    return score

study.optimize(objective, n_trials=10)

[I 2023-02-12 19:27:08,112] Trial 0 finished with value: 0.584 and parameters: {'tfidf': False, 'normalization': True, 'scaling': False}. Best is trial 0 with value: 0.584.


Trial 0 finished with value: 0.584 and parameters: {'tfidf': False, 'normalization': True, 'scaling': False}. Best is trial 0 with value: 0.584.


[I 2023-02-12 19:27:42,693] Trial 1 finished with value: 0.586 and parameters: {'tfidf': False, 'normalization': False, 'scaling': True}. Best is trial 0 with value: 0.584.


Trial 1 finished with value: 0.586 and parameters: {'tfidf': False, 'normalization': False, 'scaling': True}. Best is trial 0 with value: 0.584.


[I 2023-02-12 19:28:08,316] Trial 2 finished with value: 0.577 and parameters: {'tfidf': True, 'normalization': True, 'scaling': True}. Best is trial 2 with value: 0.577.


...

Trial 7 finished with value: 0.582 and parameters: {'tfidf': True, 'normalization': True, 'scaling': False}. Best is trial 2 with value: 0.577.


[I 2023-02-12 19:38:38,708] Trial 8 finished with value: 0.587 and parameters: {'tfidf': False, 'normalization': False, 'scaling': False}. Best is trial 2 with value: 0.577.


Trial 8 finished with value: 0.587 and parameters: {'tfidf': False, 'normalization': False, 'scaling': False}. Best is trial 2 with value: 0.577.


[I 2023-02-12 19:39:03,423] Trial 9 finished with value: 0.578 and parameters: {'tfidf': True, 'normalization': True, 'scaling': True}. Best is trial 2 with value: 0.577.


Trial 9 finished with value: 0.578 and parameters: {'tfidf': True, 'normalization': True, 'scaling': True}. Best is trial 2 with value: 0.577.


In [9]:
study.trials_dataframe().sort_values(by="value").head(10)

,number,value,datetime_start,datetime_complete,duration,params_normalization,params_scaling,params_tfidf,state
2,2,0.577,2023-02-12 19:27:42.697463,2023-02-12 19:28:08.303165,0 days 00:00:25.605702,True,True,True,COMPLETE
9,9,0.578,2023-02-12 19:38:38.712430,2023-02-12 19:39:03.410964,0 days 00:00:24.698534,True,True,True,COMPLETE
6,6,0.579,2023-02-12 19:32:29.077328,2023-02-12 19:34:42.318137,0 days 00:02:13.240809,False,False,False,COMPLETE
3,3,0.581,2023-02-12 19:28:08.320091,2023-02-12 19:28:32.611529,0 days 00:00:24.291438,True,True,False,COMPLETE
5,5,0.581,2023-02-12 19:30:41.877848,2023-02-12 19:32:29.060359,0 days 00:01:47.182511,True,False,True,COMPLETE
7,7,0.582,2023-02-12 19:34:42.333613,2023-02-12 19:36:25.262902,0 days 00:01:42.929289,True,False,True,COMPLETE
0,0,0.584,2023-02-12 19:25:00.574415,2023-02-12 19:27:08.093968,0 days 00:02:07.519553,True,False,False,COMPLETE
1,1,0.586,2023-02-12 19:27:08.116439,2023-02-12 19:27:42.680967,0 days 00:00:34.564528,False,True,False,COMPLETE
8,8,0.587,2023-02-12 19:36:25.280903,2023-02-12 19:38:38.695933,0 days 00:02:13.415030,False,False,False,COMPLETE
4,4,0.590,2023-02-12 19:28:32.628512,2023-02-12 19:30:41.862347,0 days 00:02:09.233835,True,False,False,COMPLETE


In [10]:
print(f"Best linear regression params: {study.best_params}, achieving val RMSE: {study.best_value}")

Best linear regression params: {'normalization': True, 'scaling': True, 'tfidf': True}, achieving val RMSE: 0.577


In [11]:
save = True
load = False

pipe_params = {
    'normalization': True,
    'scaling': True,
    'tfidf': True,
}

if load:
    pipe = joblib.load(f"{exp_dir}/linear/pipe")
else:
    regressor = LinearRegression()
    pipe = sklearn_regressors.train_pipe(regressor, X_train, y_train, **pipe_params)

    if save:
        additional_attributes = {
            "pipe_params": pipe_params,
            "test_programs": test_programs,
        }

        joblib.dump(pipe, f"{exp_dir}/linear/pipe")
        with open(f"{exp_dir}/linear/additional_attributes.json", "w") as file:
            json.dump(additional_attributes, file, indent=4)

In [12]:
maes = {}
prog_energy = {}

for test_program in test_programs:
    prog_df = test_df[test_df.program_name == test_program]
    prog_bbs = np.array([" ".join(bb) for bb in prog_df.bb.tolist()])
    test_preds = pipe.predict(prog_bbs)
    true_energies = prog_df.energy.values
    
    maes[test_program] = mean_absolute_error(true_energies, test_preds)
    prog_energy[test_program] = {
        "true_energy": sum(true_energies),
        "pred_energy": sum(test_preds),
    }

print("Mean absolute error for test programs")
print(f"{maes}\n")

print("Total energies for test programs")
print(prog_energy)

Mean absolute error for test programs
{'faa': 0.8564875429972671, 'quicksort': 0.2580165094097379, 'declare': 0.42751488030627116}

Total energies for test programs
{'faa': {'true_energy': 1593.1851865544543, 'pred_energy': 581.280986254267}, 'quicksort': {'true_energy': 4246.078264105911, 'pred_energy': 7350.651970431555}, 'declare': {'true_energy': 6382.83618144921, 'pred_energy': 3866.6772622437725}}


## Lasso

Linear regression with L1 regularization. Tune:
* alpha (L1 regularization term).

In [13]:
study = optuna.create_study(study_name="lasso-regression", storage=storage_name, load_if_exists=True, direction="minimize")
study.set_user_attr("Loss", "RMSE")

[I 2023-02-13 10:08:44,491] Using an existing study with name 'lasso-regression' instead of creating a new one.


Using an existing study with name 'lasso-regression' instead of creating a new one.


In [13]:
def objective(trial):

    alpha = trial.suggest_float("alpha", 0.1, 10.0)
    tfidf = trial.suggest_categorical("tfidf", [True, False])
    normalization = trial.suggest_categorical("normalization", [True, False])
    scaling = trial.suggest_categorical("scaling", [True, False])

    regressor = Lasso(alpha=alpha)
    score = sklearn_regressors.evaluate_regressor(
        regressor=regressor,
        X=X_train,
        y=y_train,
        tfidf=tfidf,
        normalization=normalization, 
        scaling=scaling
    )

    return score

study.optimize(objective, n_trials=20)

[I 2023-02-12 19:39:18,857] Trial 0 finished with value: 0.698 and parameters: {'alpha': 4.814427097179507, 'tfidf': False, 'normalization': True, 'scaling': False}. Best is trial 0 with value: 0.698.


Trial 0 finished with value: 0.698 and parameters: {'alpha': 4.814427097179507, 'tfidf': False, 'normalization': True, 'scaling': False}. Best is trial 0 with value: 0.698.


[I 2023-02-12 19:39:33,720] Trial 1 finished with value: 0.711 and parameters: {'alpha': 1.88409913571284, 'tfidf': False, 'normalization': True, 'scaling': False}. Best is trial 0 with value: 0.698.


Trial 1 finished with value: 0.711 and parameters: {'alpha': 1.88409913571284, 'tfidf': False, 'normalization': True, 'scaling': False}. Best is trial 0 with value: 0.698.


[I 2023-02-12 19:39:48,553] Trial 2 finished with value: 0.703 and parameters: {'alpha': 1.9321236682453649, 'tfidf': False, 'normalization': True, 'scaling': False}. Best is trial 0 with value: 0.698.


...

Trial 17 finished with value: 0.71 and parameters: {'alpha': 3.3599300135642705, 'tfidf': True, 'normalization': True, 'scaling': False}. Best is trial 0 with value: 0.698.


[I 2023-02-12 19:43:49,049] Trial 18 finished with value: 0.701 and parameters: {'alpha': 6.1804517447271206, 'tfidf': False, 'normalization': True, 'scaling': True}. Best is trial 0 with value: 0.698.


Trial 18 finished with value: 0.701 and parameters: {'alpha': 6.1804517447271206, 'tfidf': False, 'normalization': True, 'scaling': True}. Best is trial 0 with value: 0.698.


[I 2023-02-12 19:44:04,215] Trial 19 finished with value: 0.705 and parameters: {'alpha': 0.5037687817231662, 'tfidf': True, 'normalization': True, 'scaling': False}. Best is trial 0 with value: 0.698.


Trial 19 finished with value: 0.705 and parameters: {'alpha': 0.5037687817231662, 'tfidf': True, 'normalization': True, 'scaling': False}. Best is trial 0 with value: 0.698.


In [14]:
study.trials_dataframe().sort_values(by="value").head(10)

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_normalization,params_scaling,params_tfidf,state
0,0,0.698,2023-02-12 19:39:03.656946,2023-02-12 19:39:18.840222,0 days 00:00:15.183276,4.814427,True,False,False,COMPLETE
8,8,0.700,2023-02-12 19:41:03.503741,2023-02-12 19:41:18.622516,0 days 00:00:15.118775,9.393990,True,False,True,COMPLETE
18,18,0.701,2023-02-12 19:43:34.047270,2023-02-12 19:43:49.035790,0 days 00:00:14.988520,6.180452,True,True,False,COMPLETE
12,12,0.702,2023-02-12 19:42:03.900519,2023-02-12 19:42:18.840543,0 days 00:00:14.940024,9.714165,True,False,False,COMPLETE
2,2,0.703,2023-02-12 19:39:33.723236,2023-02-12 19:39:48.541263,0 days 00:00:14.818027,1.932124,True,False,False,COMPLETE
14,14,0.704,2023-02-12 19:42:34.039209,2023-02-12 19:42:49.161452,0 days 00:00:15.122243,8.460036,True,False,True,COMPLETE
13,13,0.704,2023-02-12 19:42:18.858010,2023-02-12 19:42:34.022210,0 days 00:00:15.164200,5.405070,True,False,True,COMPLETE
4,4,0.704,2023-02-12 19:40:03.481276,2023-02-12 19:40:18.072702,0 days 00:00:14.591426,6.415501,False,False,False,COMPLETE
5,5,0.704,2023-02-12 19:40:18.088171,2023-02-12 19:40:33.238692,0 days 00:00:15.150521,3.699033,False,True,True,COMPLETE
19,19,0.705,2023-02-12 19:43:49.052268,2023-02-12 19:44:04.203400,0 days 00:00:15.151132,0.503769,True,False,True,COMPLETE


In [15]:
print(f"Best lasso regression params: {study.best_params}, achieving val RMSE: {study.best_value}")

Best lasso regression params: {'alpha': 4.814427097179507, 'normalization': True, 'scaling': False, 'tfidf': False}, achieving val RMSE: 0.698


In [16]:
save = True
load = False

regressor_params = {"alpha": 4.81}
pipe_params = {
    'normalization': True,
    'scaling': False,
    'tfidf': False,
}

if load:
    pipe = joblib.load(f"{exp_dir}/lasso/pipe")
else:
    regressor = Lasso(**regressor_params)
    pipe = sklearn_regressors.train_pipe(regressor, X_train, y_train, **pipe_params)

    if save:
        additional_attributes = {
            "regressor_params": regressor_params,
            "pipe_params": pipe_params,
            "test_programs": test_programs,
        }

        joblib.dump(pipe, f"{exp_dir}/lasso/pipe")
        with open(f"{exp_dir}/lasso/additional_attributes.json", "w") as file:
            json.dump(additional_attributes, file, indent=4)

In [17]:
maes = {}
prog_energy = {}

for test_program in test_programs:
    prog_df = test_df[test_df.program_name == test_program]
    prog_bbs = np.array([" ".join(bb) for bb in prog_df.bb.tolist()])
    test_preds = pipe.predict(prog_bbs)
    true_energies = prog_df.energy.values
    
    maes[test_program] = mean_absolute_error(true_energies, test_preds)
    prog_energy[test_program] = {
        "true_energy": sum(true_energies),
        "pred_energy": sum(test_preds),
    }

print("Mean absolute error for test programs")
print(f"{maes}\n")

print("Total energies for test programs")
print(prog_energy)

Mean absolute error for test programs
{'faa': 0.9109510074307886, 'quicksort': 0.31704603813120613, 'declare': 0.4567457793792269}

Total energies for test programs
{'faa': {'true_energy': 1593.1851865544543, 'pred_energy': 475.4874028864406}, 'quicksort': {'true_energy': 4246.078264105911, 'pred_energy': 6599.130136312516}, 'declare': {'true_energy': 6382.83618144921, 'pred_energy': 3989.757491321512}}


## Ridge

Linear regression with L2 regularization. Tune:
* alpha (L2 regularization term).

In [18]:
study = optuna.create_study(study_name="ridge-regression", storage=storage_name, load_if_exists=True, direction="minimize")
study.set_user_attr("Loss", "RMSE")

[I 2023-02-13 10:08:59,535] Using an existing study with name 'ridge-regression' instead of creating a new one.


Using an existing study with name 'ridge-regression' instead of creating a new one.


In [24]:
def objective(trial):

    alpha = trial.suggest_float("alpha", 0.1, 10.0)
    tfidf = trial.suggest_categorical("tfidf", [True, False])
    normalization = trial.suggest_categorical("normalization", [True, False])
    scaling = trial.suggest_categorical("scaling", [True, False])

    regressor = Ridge(alpha=alpha)
    score = sklearn_regressors.evaluate_regressor(
        regressor=regressor,
        X=X_train,
        y=y_train,
        tfidf=tfidf,
        normalization=normalization, 
        scaling=scaling
    )

    return score

study.optimize(objective, n_trials=20)

[I 2023-02-13 00:17:52,008] Trial 0 finished with value: 0.571 and parameters: {'alpha': 4.580125801925755, 'tfidf': True, 'normalization': True, 'scaling': False}. Best is trial 0 with value: 0.571.


Trial 0 finished with value: 0.571 and parameters: {'alpha': 4.580125801925755, 'tfidf': True, 'normalization': True, 'scaling': False}. Best is trial 0 with value: 0.571.


[I 2023-02-13 00:18:07,805] Trial 1 finished with value: 0.578 and parameters: {'alpha': 0.9453533393144836, 'tfidf': True, 'normalization': True, 'scaling': True}. Best is trial 0 with value: 0.571.


Trial 1 finished with value: 0.578 and parameters: {'alpha': 0.9453533393144836, 'tfidf': True, 'normalization': True, 'scaling': True}. Best is trial 0 with value: 0.571.


[I 2023-02-13 00:18:25,037] Trial 2 finished with value: 0.586 and parameters: {'alpha': 4.883188382960704, 'tfidf': False, 'normalization': False, 'scaling': True}. Best is trial 0 with value: 0.571.


...

Trial 17 finished with value: 0.581 and parameters: {'alpha': 1.8123402010923577, 'tfidf': False, 'normalization': False, 'scaling': False}. Best is trial 11 with value: 0.569.


[I 2023-02-13 00:23:18,244] Trial 18 finished with value: 0.574 and parameters: {'alpha': 3.814716619114115, 'tfidf': False, 'normalization': False, 'scaling': False}. Best is trial 11 with value: 0.569.


Trial 18 finished with value: 0.574 and parameters: {'alpha': 3.814716619114115, 'tfidf': False, 'normalization': False, 'scaling': False}. Best is trial 11 with value: 0.569.


[I 2023-02-13 00:23:39,486] Trial 19 finished with value: 0.575 and parameters: {'alpha': 1.9522098761230005, 'tfidf': False, 'normalization': False, 'scaling': False}. Best is trial 11 with value: 0.569.


Trial 19 finished with value: 0.575 and parameters: {'alpha': 1.9522098761230005, 'tfidf': False, 'normalization': False, 'scaling': False}. Best is trial 11 with value: 0.569.


In [19]:
study.trials_dataframe().sort_values(by="value").head(10)

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_normalization,params_scaling,params_tfidf,state
12,12,0.569,2023-02-13 00:20:52.494345,2023-02-13 00:21:12.844826,0 days 00:00:20.350481,2.705393,False,False,False,COMPLETE
11,11,0.569,2023-02-13 00:20:32.332712,2023-02-13 00:20:52.477348,0 days 00:00:20.144636,2.964135,False,False,False,COMPLETE
0,0,0.571,2023-02-13 00:17:35.266633,2023-02-13 00:17:51.989490,0 days 00:00:16.722857,4.580126,True,False,True,COMPLETE
10,10,0.571,2023-02-13 00:20:11.519570,2023-02-13 00:20:32.313745,0 days 00:00:20.794175,2.873149,False,False,False,COMPLETE
5,5,0.572,2023-02-13 00:18:55.911532,2023-02-13 00:19:11.024032,0 days 00:00:15.112500,8.685663,True,True,True,COMPLETE
15,15,0.573,2023-02-13 00:21:57.422958,2023-02-13 00:22:17.729151,0 days 00:00:20.306193,2.875274,False,False,False,COMPLETE
14,14,0.574,2023-02-13 00:21:33.089674,2023-02-13 00:21:57.407457,0 days 00:00:24.317783,0.245051,False,False,False,COMPLETE
4,4,0.574,2023-02-13 00:18:40.868462,2023-02-13 00:18:55.895532,0 days 00:00:15.027070,7.909947,True,True,True,COMPLETE
18,18,0.574,2023-02-13 00:22:58.231718,2023-02-13 00:23:18.231779,0 days 00:00:20.000061,3.814717,False,False,False,COMPLETE
16,16,0.575,2023-02-13 00:22:17.745150,2023-02-13 00:22:37.604546,0 days 00:00:19.859396,3.469536,False,False,False,COMPLETE


In [20]:
print(f"Best ridge regression params: {study.best_params}, achieving val RMSE: {study.best_value}")

Best ridge regression params: {'alpha': 2.964135035227021, 'normalization': False, 'scaling': False, 'tfidf': False}, achieving val RMSE: 0.569


In [21]:
save = False
load = True

regressor_params = {"alpha": 2.96}
pipe_params = {
    'normalization': False,
    'scaling': False,
    'tfidf': False,
}

if load:
    pipe = joblib.load(f"{exp_dir}/ridge/pipe")
else:
    regressor = Ridge(**regressor_params)
    pipe = sklearn_regressors.train_pipe(regressor, X_train, y_train, **pipe_params)

    if save:
        additional_attributes = {
            "regressor_params": regressor_params,
            "pipe_params": pipe_params,
            "test_programs": test_programs,
        }

        joblib.dump(pipe, f"{exp_dir}/ridge/pipe")
        with open(f"{exp_dir}/ridge/additional_attributes.json", "w") as file:
            json.dump(additional_attributes, file, indent=4)

In [22]:
maes = {}
prog_energy = {}

for test_program in test_programs:
    prog_df = test_df[test_df.program_name == test_program]
    prog_bbs = np.array([" ".join(bb) for bb in prog_df.bb.tolist()])
    test_preds = pipe.predict(prog_bbs)
    true_energies = prog_df.energy.values
    
    maes[test_program] = mean_absolute_error(true_energies, test_preds)
    prog_energy[test_program] = {
        "true_energy": sum(true_energies),
        "pred_energy": sum(test_preds),
    }

print("Mean absolute error for test programs")
print(f"{maes}\n")

print("Total energies for test programs")
print(prog_energy)

Mean absolute error for test programs
{'faa': 0.8569234326811495, 'quicksort': 0.25804184569469063, 'declare': 0.4274168206139757}

Total energies for test programs
{'faa': {'true_energy': 1593.1851865544543, 'pred_energy': 582.1524586769206}, 'quicksort': {'true_energy': 4246.078264105911, 'pred_energy': 7351.804519804801}, 'declare': {'true_energy': 6382.83618144921, 'pred_energy': 3867.893760943608}}


## ElasticNet

Linear regression with both L1 and L2 regularization. Tune
* alpha
* l1_ratio. 

In [23]:
study = optuna.create_study(study_name="elasticnet-regression", storage=storage_name, load_if_exists=True, direction="minimize")
study.set_user_attr("Loss", "RMSE")

[I 2023-02-13 10:09:24,875] Using an existing study with name 'elasticnet-regression' instead of creating a new one.


Using an existing study with name 'elasticnet-regression' instead of creating a new one.


In [21]:
def objective(trial):

    alpha = trial.suggest_float("alpha", 0.1, 10.0)
    l1_ratio = trial.suggest_float("l1_ratio", 0.1, 0.9)
    tfidf = trial.suggest_categorical("tfidf", [True, False])
    normalization = trial.suggest_categorical("normalization", [True, False])
    scaling = trial.suggest_categorical("scaling", [True, False])

    regressor = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
    score = sklearn_regressors.evaluate_regressor(
        regressor=regressor,
        X=X_train,
        y=y_train,
        tfidf=tfidf,
        normalization=normalization, 
        scaling=scaling
    )

    return score

study.optimize(objective, n_trials=20)

[I 2023-02-12 20:04:13,106] Trial 0 finished with value: 0.712 and parameters: {'alpha': 7.6767592564378555, 'l1_ratio': 0.7947754314368692, 'tfidf': False, 'normalization': True, 'scaling': False}. Best is trial 0 with value: 0.712.


Trial 0 finished with value: 0.712 and parameters: {'alpha': 7.6767592564378555, 'l1_ratio': 0.7947754314368692, 'tfidf': False, 'normalization': True, 'scaling': False}. Best is trial 0 with value: 0.712.


[I 2023-02-12 20:04:27,854] Trial 1 finished with value: 0.703 and parameters: {'alpha': 5.691982379501282, 'l1_ratio': 0.8611016476247246, 'tfidf': False, 'normalization': True, 'scaling': False}. Best is trial 1 with value: 0.703.


Trial 1 finished with value: 0.703 and parameters: {'alpha': 5.691982379501282, 'l1_ratio': 0.8611016476247246, 'tfidf': False, 'normalization': True, 'scaling': False}. Best is trial 1 with value: 0.703.


[I 2023-02-12 20:04:42,789] Trial 2 finished with value: 0.712 and parameters: {'alpha': 3.0480953980362853, 'l1_ratio': 0.777522900069485, 'tfidf': True, 'normalization': False, 'scaling': False}. Best is trial 1 with value: 0.703.


...

Trial 17 finished with value: 0.706 and parameters: {'alpha': 1.379902294338415, 'l1_ratio': 0.23948946362395285, 'tfidf': False, 'normalization': False, 'scaling': True}. Best is trial 15 with value: 0.617.


[I 2023-02-12 20:10:35,635] Trial 18 finished with value: 0.708 and parameters: {'alpha': 4.025708596980282, 'l1_ratio': 0.10381172088245072, 'tfidf': True, 'normalization': False, 'scaling': True}. Best is trial 15 with value: 0.617.


Trial 18 finished with value: 0.708 and parameters: {'alpha': 4.025708596980282, 'l1_ratio': 0.10381172088245072, 'tfidf': True, 'normalization': False, 'scaling': True}. Best is trial 15 with value: 0.617.


[I 2023-02-12 20:11:32,678] Trial 19 finished with value: 0.606 and parameters: {'alpha': 0.11242693435738552, 'l1_ratio': 0.23653131327938587, 'tfidf': False, 'normalization': False, 'scaling': True}. Best is trial 19 with value: 0.606.


Trial 19 finished with value: 0.606 and parameters: {'alpha': 0.11242693435738552, 'l1_ratio': 0.23653131327938587, 'tfidf': False, 'normalization': False, 'scaling': True}. Best is trial 19 with value: 0.606.


In [24]:
study.trials_dataframe().sort_values(by="value").head(10)

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_l1_ratio,params_normalization,params_scaling,params_tfidf,state
19,19,0.606,2023-02-12 20:10:35.639328,2023-02-12 20:11:32.665827,0 days 00:00:57.026499,0.112427,0.236531,False,True,False,COMPLETE
15,15,0.617,2023-02-12 20:09:05.075636,2023-02-12 20:09:50.259511,0 days 00:00:45.183875,0.309089,0.136059,False,True,False,COMPLETE
14,14,0.625,2023-02-12 20:08:20.431011,2023-02-12 20:09:05.059881,0 days 00:00:44.628870,0.293769,0.170803,False,True,False,COMPLETE
13,13,0.626,2023-02-12 20:07:40.196028,2023-02-12 20:08:20.414043,0 days 00:00:40.218015,0.400910,0.176536,False,True,False,COMPLETE
11,11,0.630,2023-02-12 20:06:43.347274,2023-02-12 20:07:25.046306,0 days 00:00:41.699032,0.159982,0.617574,False,True,False,COMPLETE
7,7,0.697,2023-02-12 20:05:42.930764,2023-02-12 20:05:57.903260,0 days 00:00:14.972496,9.692301,0.686528,True,False,False,COMPLETE
9,9,0.699,2023-02-12 20:06:13.302679,2023-02-12 20:06:28.116305,0 days 00:00:14.813626,1.239349,0.434419,False,False,False,COMPLETE
10,10,0.701,2023-02-12 20:06:28.132778,2023-02-12 20:06:43.331306,0 days 00:00:15.198528,9.878726,0.597755,True,True,False,COMPLETE
5,5,0.703,2023-02-12 20:05:12.488259,2023-02-12 20:05:27.375760,0 days 00:00:14.887501,5.673337,0.391687,False,True,False,COMPLETE
1,1,0.703,2023-02-12 20:04:13.109187,2023-02-12 20:04:27.838393,0 days 00:00:14.729206,5.691982,0.861102,True,False,False,COMPLETE


In [25]:
print(f"Best ElasticNET regression params: {study.best_params}, achieving val RMSE: {study.best_value}")

Best ElasticNET regression params: {'alpha': 0.11242693435738552, 'l1_ratio': 0.23653131327938587, 'normalization': False, 'scaling': True, 'tfidf': False}, achieving val RMSE: 0.606


In [26]:
save = False
load = True

regressor_params = {"alpha": 0.11, "l1_ratio": 0.24}
pipe_params = {
    'normalization': False,
    'scaling': True,
    'tfidf': False,
}

if load:
    pipe = joblib.load(f"{exp_dir}/elasticnet/pipe")
else:
    regressor = ElasticNet(**regressor_params)
    pipe = sklearn_regressors.train_pipe(regressor, X_train, y_train, **pipe_params)

    if save:
        additional_attributes = {
            "regressor_params": regressor_params,
            "pipe_params": pipe_params,
            "test_programs": test_programs,
        }

        joblib.dump(pipe, f"{exp_dir}/elasticnet/pipe")
        with open(f"{exp_dir}/elasticnet/additional_attributes.json", "w") as file:
            json.dump(additional_attributes, file, indent=4)

In [27]:
maes = {}
prog_energy = {}

for test_program in test_programs:
    prog_df = test_df[test_df.program_name == test_program]
    prog_bbs = np.array([" ".join(bb) for bb in prog_df.bb.tolist()])
    test_preds = pipe.predict(prog_bbs)
    true_energies = prog_df.energy.values
    
    maes[test_program] = mean_absolute_error(true_energies, test_preds)
    prog_energy[test_program] = {
        "true_energy": sum(true_energies),
        "pred_energy": sum(test_preds),
    }

print("Mean absolute error for test programs")
print(f"{maes}\n")

print("Total energies for test programs")
print(prog_energy)

Mean absolute error for test programs
{'faa': 0.89147165008725, 'quicksort': 0.25386099978710697, 'declare': 0.43216138903587825}

Total energies for test programs
{'faa': {'true_energy': 1593.1851865544543, 'pred_energy': 500.42793924663556}, 'quicksort': {'true_energy': 4246.078264105911, 'pred_energy': 6705.826639080588}, 'declare': {'true_energy': 6382.83618144921, 'pred_energy': 3819.1135006007057}}


## SGD Regression

Stochastic Gradient Descent Regression using either of the above reguliration techniques. Tune:
* penalty method
* alpha
* l1_ratio (if elasticnet penalty).

In [28]:
study = optuna.create_study(study_name="SGD-regression", storage=storage_name, load_if_exists=True, direction="minimize")
study.set_user_attr("Loss", "RMSE")

[I 2023-02-13 10:09:42,950] Using an existing study with name 'SGD-regression' instead of creating a new one.


Using an existing study with name 'SGD-regression' instead of creating a new one.


In [25]:
def objective(trial):

    penalty = trial.suggest_categorical("penalty", ["l1", "l2", "elasticnet"])
    alpha = trial.suggest_float("alpha", 0.1, 10.0)
    l1_ratio = trial.suggest_float("l1_ratio", 0.1, 0.1)
    tfidf = trial.suggest_categorical("tfidf", [True, False])
    normalization = trial.suggest_categorical("normalization", [True, False])
    scaling = trial.suggest_categorical("scaling", [True, False])

    regressor = SGDRegressor(penalty=penalty, alpha=alpha, l1_ratio=l1_ratio)
    score = sklearn_regressors.evaluate_regressor(
        regressor=regressor,
        X=X_train,
        y=y_train,
        tfidf=tfidf,
        normalization=normalization, 
        scaling=scaling
    )

    return score

study.optimize(objective, n_trials=40)

[I 2023-02-12 20:11:48,285] Trial 0 finished with value: 0.675 and parameters: {'penalty': 'l2', 'alpha': 8.126047195718966, 'l1_ratio': 0.1, 'tfidf': True, 'normalization': False, 'scaling': True}. Best is trial 0 with value: 0.675.


Trial 0 finished with value: 0.675 and parameters: {'penalty': 'l2', 'alpha': 8.126047195718966, 'l1_ratio': 0.1, 'tfidf': True, 'normalization': False, 'scaling': True}. Best is trial 0 with value: 0.675.


[I 2023-02-12 20:12:03,083] Trial 1 finished with value: 0.713 and parameters: {'penalty': 'l1', 'alpha': 8.840943103159544, 'l1_ratio': 0.1, 'tfidf': True, 'normalization': True, 'scaling': False}. Best is trial 0 with value: 0.675.


Trial 1 finished with value: 0.713 and parameters: {'penalty': 'l1', 'alpha': 8.840943103159544, 'l1_ratio': 0.1, 'tfidf': True, 'normalization': True, 'scaling': False}. Best is trial 0 with value: 0.675.


[I 2023-02-12 20:12:17,886] Trial 2 finished with value: 1773240610374.694 and parameters: {'penalty': 'l1', 'alpha': 0.24060479728254555, 'l1_ratio': 0.1, 'tfidf': False, 'normalization': True, 'scaling': True}. Best is trial 0 with value: 0.675.


...

Trial 37 finished with value: 0.706 and parameters: {'penalty': 'l1', 'alpha': 2.50972994638051, 'l1_ratio': 0.1, 'tfidf': False, 'normalization': False, 'scaling': False}. Best is trial 27 with value: 0.646.


[I 2023-02-12 20:21:06,646] Trial 38 finished with value: 0.654 and parameters: {'penalty': 'l2', 'alpha': 2.7617317641419845, 'l1_ratio': 0.1, 'tfidf': False, 'normalization': False, 'scaling': False}. Best is trial 27 with value: 0.646.


Trial 38 finished with value: 0.654 and parameters: {'penalty': 'l2', 'alpha': 2.7617317641419845, 'l1_ratio': 0.1, 'tfidf': False, 'normalization': False, 'scaling': False}. Best is trial 27 with value: 0.646.


[I 2023-02-12 20:21:20,906] Trial 39 finished with value: 0.703 and parameters: {'penalty': 'l2', 'alpha': 3.3353359835785104, 'l1_ratio': 0.1, 'tfidf': True, 'normalization': False, 'scaling': False}. Best is trial 27 with value: 0.646.


Trial 39 finished with value: 0.703 and parameters: {'penalty': 'l2', 'alpha': 3.3353359835785104, 'l1_ratio': 0.1, 'tfidf': True, 'normalization': False, 'scaling': False}. Best is trial 27 with value: 0.646.


In [29]:
study.trials_dataframe().sort_values(by="value").head(10)

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_l1_ratio,params_normalization,params_penalty,params_scaling,params_tfidf,state
33,33,0.646,2023-02-12 20:19:40.921652,2023-02-12 20:19:55.090151,0 days 00:00:14.168499,4.324908,0.1,False,l2,False,False,COMPLETE
27,27,0.646,2023-02-12 20:18:15.011229,2023-02-12 20:18:29.342369,0 days 00:00:14.331140,4.394182,0.1,False,l2,False,False,COMPLETE
36,36,0.649,2023-02-12 20:20:23.841917,2023-02-12 20:20:38.031959,0 days 00:00:14.190042,2.823420,0.1,False,l2,False,False,COMPLETE
38,38,0.654,2023-02-12 20:20:52.464274,2023-02-12 20:21:06.633232,0 days 00:00:14.168958,2.761732,0.1,False,l2,False,False,COMPLETE
32,32,0.654,2023-02-12 20:19:26.693659,2023-02-12 20:19:40.906652,0 days 00:00:14.212993,4.428107,0.1,False,l2,False,False,COMPLETE
23,23,0.655,2023-02-12 20:17:18.110064,2023-02-12 20:17:32.220056,0 days 00:00:14.109992,6.133459,0.1,False,l2,False,False,COMPLETE
31,31,0.658,2023-02-12 20:19:12.543496,2023-02-12 20:19:26.676186,0 days 00:00:14.132690,6.027586,0.1,False,l2,False,False,COMPLETE
17,17,0.660,2023-02-12 20:15:51.947395,2023-02-12 20:16:06.275483,0 days 00:00:14.328088,5.391655,0.1,False,l2,False,False,COMPLETE
24,24,0.661,2023-02-12 20:17:32.235063,2023-02-12 20:17:46.365057,0 days 00:00:14.129994,5.782303,0.1,False,l2,False,False,COMPLETE
30,30,0.662,2023-02-12 20:18:58.337785,2023-02-12 20:19:12.528499,0 days 00:00:14.190714,6.127276,0.1,False,l2,False,False,COMPLETE


In [30]:
print(f"Best SGD regression params: {study.best_params}, achieving val RMSE: {study.best_value}")

Best SGD regression params: {'alpha': 4.394181615429851, 'l1_ratio': 0.1, 'normalization': False, 'penalty': 'l2', 'scaling': False, 'tfidf': False}, achieving val RMSE: 0.646


In [31]:
save = False
load = True

regressor_params = {"alpha": 4.39, "l1_ratio": 0.1, "penalty": "l2"}
pipe_params = {
    'normalization': False,
    'scaling': False,
    'tfidf': False,
}

if load:
    pipe = joblib.load(f"{exp_dir}/sgd/pipe")
else:
    regressor = SGDRegressor(**regressor_params)
    pipe = sklearn_regressors.train_pipe(regressor, X_train, y_train, **pipe_params)

    if save:
        additional_attributes = {
            "regressor_params": regressor_params,
            "pipe_params": pipe_params,
            "test_programs": test_programs,
        }

        joblib.dump(pipe, f"{exp_dir}/sgd/pipe")
        with open(f"{exp_dir}/sgd/additional_attributes.json", "w") as file:
            json.dump(additional_attributes, file, indent=4)

In [32]:
maes = {}
prog_energy = {}

for test_program in test_programs:
    prog_df = test_df[test_df.program_name == test_program]
    prog_bbs = np.array([" ".join(bb) for bb in prog_df.bb.tolist()])
    test_preds = pipe.predict(prog_bbs)
    true_energies = prog_df.energy.values
    
    maes[test_program] = mean_absolute_error(true_energies, test_preds)
    prog_energy[test_program] = {
        "true_energy": sum(true_energies),
        "pred_energy": sum(test_preds),
    }

print("Mean absolute error for test programs")
print(f"{maes}\n")

print("Total energies for test programs")
print(prog_energy)

Mean absolute error for test programs
{'faa': 0.9330119034821376, 'quicksort': 0.25573919607704576, 'declare': 0.4441559337023836}

Total energies for test programs
{'faa': {'true_energy': 1593.1851865544543, 'pred_energy': 456.1074803135414}, 'quicksort': {'true_energy': 4246.078264105911, 'pred_energy': 6264.6491942112025}, 'declare': {'true_energy': 6382.83618144921, 'pred_energy': 3514.0047917998977}}


## SVR

Support Vector Regression. Tune:
* kernel
* gamma
* C

In [8]:
study = optuna.create_study(study_name="SVR-regression", storage=storage_name, load_if_exists=True, direction="minimize")
study.set_user_attr("Loss", "RMSE")

[I 2023-02-13 15:33:41,824] Using an existing study with name 'SVR-regression' instead of creating a new one.


Using an existing study with name 'SVR-regression' instead of creating a new one.


In [10]:
def objective(trial):

    kernel = trial.suggest_categorical("kernel", ["rbf", "linear"])
    C = trial.suggest_float("C", 0.1, 10.0)
    gamma = trial.suggest_float("gamma", 0.1, 0.1)
    tfidf = trial.suggest_categorical("tfidf", [True, False])
    normalization = trial.suggest_categorical("normalization", [True, False])
    scaling = trial.suggest_categorical("scaling", [True, False])

    regressor = SVR(kernel=kernel, C=C, gamma=gamma)
    score = sklearn_regressors.evaluate_regressor(
        regressor=regressor,
        X=X_train[:10000],
        y=y_train[:10000],
        tfidf=tfidf,
        normalization=normalization, 
        scaling=scaling
    )

    return score

study.optimize(objective, n_trials=40)

[I 2023-02-13 00:57:53,743] Trial 0 finished with value: 0.604 and parameters: {'kernel': 'linear', 'C': 5.208669556171662, 'gamma': 0.1, 'tfidf': False, 'normalization': False, 'scaling': False}. Best is trial 0 with value: 0.604.


Trial 0 finished with value: 0.604 and parameters: {'kernel': 'linear', 'C': 5.208669556171662, 'gamma': 0.1, 'tfidf': False, 'normalization': False, 'scaling': False}. Best is trial 0 with value: 0.604.


[I 2023-02-13 01:22:24,755] Trial 1 finished with value: 0.614 and parameters: {'kernel': 'linear', 'C': 4.936226669403213, 'gamma': 0.1, 'tfidf': True, 'normalization': False, 'scaling': True}. Best is trial 0 with value: 0.604.


Trial 1 finished with value: 0.614 and parameters: {'kernel': 'linear', 'C': 4.936226669403213, 'gamma': 0.1, 'tfidf': True, 'normalization': False, 'scaling': True}. Best is trial 0 with value: 0.604.


[I 2023-02-13 01:22:34,089] Trial 2 finished with value: 0.635 and parameters: {'kernel': 'rbf', 'C': 4.441320775205987, 'gamma': 0.1, 'tfidf': True, 'normalization': True, 'scaling': True}. Best is trial 0 with value: 0.604.


...

Trial 37 finished with value: 0.618 and parameters: {'kernel': 'rbf', 'C': 4.619650988104117, 'gamma': 0.1, 'tfidf': False, 'normalization': False, 'scaling': True}. Best is trial 20 with value: 0.554.


[I 2023-02-13 08:19:33,769] Trial 38 finished with value: 0.547 and parameters: {'kernel': 'rbf', 'C': 7.399229326147877, 'gamma': 0.1, 'tfidf': False, 'normalization': False, 'scaling': False}. Best is trial 38 with value: 0.547.


Trial 38 finished with value: 0.547 and parameters: {'kernel': 'rbf', 'C': 7.399229326147877, 'gamma': 0.1, 'tfidf': False, 'normalization': False, 'scaling': False}. Best is trial 38 with value: 0.547.


[I 2023-02-13 08:19:41,676] Trial 39 finished with value: 0.597 and parameters: {'kernel': 'rbf', 'C': 6.6553410056005475, 'gamma': 0.1, 'tfidf': False, 'normalization': False, 'scaling': False}. Best is trial 38 with value: 0.547.


Trial 39 finished with value: 0.597 and parameters: {'kernel': 'rbf', 'C': 6.6553410056005475, 'gamma': 0.1, 'tfidf': False, 'normalization': False, 'scaling': False}. Best is trial 38 with value: 0.547.


In [9]:
study.trials_dataframe().sort_values(by="value").head(10)

,number,value,datetime_start,datetime_complete,duration,params_C,params_gamma,params_kernel,params_normalization,params_scaling,params_tfidf,state
38,38,0.547,2023-02-13 08:19:25.985788,2023-02-13 08:19:33.755813,0 days 00:00:07.770025,7.399229,0.1,rbf,False,False,False,COMPLETE
20,20,0.554,2023-02-13 06:10:36.342999,2023-02-13 06:10:45.600992,0 days 00:00:09.257993,9.159041,0.1,rbf,False,True,False,COMPLETE
25,25,0.557,2023-02-13 06:11:22.816078,2023-02-13 06:11:32.107077,0 days 00:00:09.290999,8.021215,0.1,rbf,False,True,False,COMPLETE
7,7,0.559,2023-02-13 01:23:12.054681,2023-02-13 01:23:21.257705,0 days 00:00:09.203024,5.455872,0.1,rbf,False,True,False,COMPLETE
8,8,0.564,2023-02-13 01:23:21.273674,2023-02-13 02:25:38.167492,0 days 01:02:16.893818,8.705073,0.1,linear,True,True,True,COMPLETE
12,12,0.564,2023-02-13 03:16:45.168369,2023-02-13 03:16:54.440032,0 days 00:00:09.271663,7.275445,0.1,rbf,True,True,False,COMPLETE
28,28,0.568,2023-02-13 06:11:49.385882,2023-02-13 06:11:58.766880,0 days 00:00:09.380998,7.814291,0.1,rbf,False,True,False,COMPLETE
6,6,0.568,2023-02-13 01:23:04.210359,2023-02-13 01:23:12.038704,0 days 00:00:07.828345,2.200025,0.1,rbf,False,False,False,COMPLETE
18,18,0.571,2023-02-13 05:35:28.589634,2023-02-13 05:35:37.904512,0 days 00:00:09.314878,5.964181,0.1,rbf,True,True,True,COMPLETE
14,14,0.575,2023-02-13 03:56:18.538148,2023-02-13 03:56:27.784050,0 days 00:00:09.245902,8.388471,0.1,rbf,False,True,True,COMPLETE


In [10]:
print(f"Best SVR regression params: {study.best_params}, achieving val RMSE: {study.best_value}")

Best SVR regression params: {'C': 7.399229326147877, 'gamma': 0.1, 'kernel': 'rbf', 'normalization': False, 'scaling': False, 'tfidf': False}, achieving val RMSE: 0.547


In [17]:
save = True
load = False

regressor_params = {"kernel": "rbf", "C": 7.4, "gamma": 0.1}
pipe_params = {
    'normalization': False,
    'scaling': False,
    'tfidf': False,
}

sample = 50000

if load:
    pipe = joblib.load(f"{exp_dir}/svr/pipe")
else:
    regressor = SVR(**regressor_params)
    pipe = sklearn_regressors.train_pipe(regressor, X_train[:sample], y_train[:sample], **pipe_params)
    if save:
        additional_attributes = {
            "regressor_params": regressor_params,
            "pipe_params": pipe_params,
            "test_programs": test_programs,
        }

        joblib.dump(pipe, f"{exp_dir}/svr/pipe")
        with open(f"{exp_dir}/svr/additional_attributes.json", "w") as file:
            json.dump(additional_attributes, file, indent=4)

In [18]:
maes = {}
prog_energy = {}

for test_program in test_programs:
    prog_df = test_df[test_df.program_name == test_program]
    prog_bbs = np.array([" ".join(bb) for bb in prog_df.bb.tolist()])
    test_preds = pipe.predict(prog_bbs)
    true_energies = prog_df.energy.values
    
    maes[test_program] = mean_absolute_error(true_energies, test_preds)
    prog_energy[test_program] = {
        "true_energy": sum(true_energies),
        "pred_energy": sum(test_preds),
    }

print("Mean absolute error for test programs")
print(f"{maes}\n")

print("Total energies for test programs")
print(prog_energy)

Mean absolute error for test programs
{'faa': 0.947999546789649, 'quicksort': 0.20917131737962452, 'declare': 0.46181885270920847}

Total energies for test programs
{'faa': {'true_energy': 1593.1851865544559, 'pred_energy': 457.6059564138028}, 'quicksort': {'true_energy': 4246.078264105905, 'pred_energy': 6267.793849637768}, 'declare': {'true_energy': 6382.836181449226, 'pred_energy': 2851.4726615175578}}


## Hist Gradient Boosting Regressor

Gradient Boosting regression for large datasets. Tune:
* learning rate
* max leaf nodes
* l2 regularization

In [8]:
study = optuna.create_study(study_name="HistGBoost-regression", storage=storage_name, load_if_exists=True, direction="minimize")
study.set_user_attr("Loss", "RMSE")

[I 2023-02-13 15:34:41,326] A new study created in RDB with name: HistGBoost-regression


A new study created in RDB with name: HistGBoost-regression


In [9]:
def objective(trial):

    learning_rate = trial.suggest_float("learning_rate", 1e-4, 0.5)
    max_leaf_nodes = trial.suggest_int("max_leaf_nodes", 10, 60)
    l2_regularization = trial.suggest_float("l2_regularization", 0.1, 10.0)
    tfidf = trial.suggest_categorical("tfidf", [True, False])
    normalization = trial.suggest_categorical("normalization", [True, False])
    scaling = trial.suggest_categorical("scaling", [True, False])

    regressor = HistGradientBoostingRegressor(learning_rate=learning_rate, max_leaf_nodes=max_leaf_nodes, l2_regularization=l2_regularization)
    score = sklearn_regressors.evaluate_regressor(
        regressor=regressor,
        X=X_train,
        y=y_train,
        tfidf=tfidf,
        normalization=normalization, 
        scaling=scaling,
        requires_dense=True,
    )

    return score

study.optimize(objective, n_trials=40)

[I 2023-02-13 15:36:56,474] Trial 0 finished with value: 0.564 and parameters: {'learning_rate': 0.2942079553055097, 'max_leaf_nodes': 16, 'l2_regularization': 5.512124349723596, 'tfidf': True, 'normalization': False, 'scaling': False}. Best is trial 0 with value: 0.564.


Trial 0 finished with value: 0.564 and parameters: {'learning_rate': 0.2942079553055097, 'max_leaf_nodes': 16, 'l2_regularization': 5.512124349723596, 'tfidf': True, 'normalization': False, 'scaling': False}. Best is trial 0 with value: 0.564.


[I 2023-02-13 15:38:53,078] Trial 1 finished with value: 0.561 and parameters: {'learning_rate': 0.30633465499519946, 'max_leaf_nodes': 13, 'l2_regularization': 8.880231849204621, 'tfidf': True, 'normalization': True, 'scaling': True}. Best is trial 1 with value: 0.561.


Trial 1 finished with value: 0.561 and parameters: {'learning_rate': 0.30633465499519946, 'max_leaf_nodes': 13, 'l2_regularization': 8.880231849204621, 'tfidf': True, 'normalization': True, 'scaling': True}. Best is trial 1 with value: 0.561.


[I 2023-02-13 15:41:52,067] Trial 2 finished with value: 0.582 and parameters: {'learning_rate': 0.017378811465498512, 'max_leaf_nodes': 31, 'l2_regularization': 0.1338996789187556, 'tfidf': True, 'normalization': False, 'scaling': True}. Best is trial 1 with value: 0.561.


...

Trial 37 finished with value: 0.565 and parameters: {'learning_rate': 0.15251306407309845, 'max_leaf_nodes': 10, 'l2_regularization': 8.029144757963675, 'tfidf': True, 'normalization': True, 'scaling': False}. Best is trial 20 with value: 0.558.


[I 2023-02-13 17:10:45,476] Trial 38 finished with value: 0.561 and parameters: {'learning_rate': 0.2270912819968195, 'max_leaf_nodes': 32, 'l2_regularization': 9.58297881151908, 'tfidf': True, 'normalization': False, 'scaling': True}. Best is trial 20 with value: 0.558.


Trial 38 finished with value: 0.561 and parameters: {'learning_rate': 0.2270912819968195, 'max_leaf_nodes': 32, 'l2_regularization': 9.58297881151908, 'tfidf': True, 'normalization': False, 'scaling': True}. Best is trial 20 with value: 0.558.


[I 2023-02-13 17:12:55,185] Trial 39 finished with value: 0.567 and parameters: {'learning_rate': 0.28839988443284753, 'max_leaf_nodes': 18, 'l2_regularization': 5.72787286872817, 'tfidf': True, 'normalization': True, 'scaling': True}. Best is trial 20 with value: 0.558.


Trial 39 finished with value: 0.567 and parameters: {'learning_rate': 0.28839988443284753, 'max_leaf_nodes': 18, 'l2_regularization': 5.72787286872817, 'tfidf': True, 'normalization': True, 'scaling': True}. Best is trial 20 with value: 0.558.


In [10]:
study.trials_dataframe().sort_values(by="value").head(10)

,number,value,datetime_start,datetime_complete,duration,params_l2_regularization,params_learning_rate,params_max_leaf_nodes,params_normalization,params_scaling,params_tfidf,state
20,20,0.558,2023-02-13 16:27:53.031437,2023-02-13 16:30:13.552935,0 days 00:02:20.521498,8.803675,0.235958,20,False,True,True,COMPLETE
23,23,0.560,2023-02-13 16:34:51.510108,2023-02-13 16:36:55.377005,0 days 00:02:03.866897,9.057125,0.315053,14,False,True,True,COMPLETE
1,1,0.561,2023-02-13 15:36:56.492012,2023-02-13 15:38:53.064597,0 days 00:01:56.572585,8.880232,0.306335,13,True,True,True,COMPLETE
38,38,0.561,2023-02-13 17:07:55.122314,2023-02-13 17:10:45.452352,0 days 00:02:50.330038,9.582979,0.227091,32,False,True,True,COMPLETE
29,29,0.561,2023-02-13 16:47:44.578250,2023-02-13 16:49:56.268404,0 days 00:02:11.690154,8.222417,0.286817,20,False,False,True,COMPLETE
7,7,0.562,2023-02-13 15:52:30.038867,2023-02-13 15:55:08.636643,0 days 00:02:38.597776,5.685665,0.457672,60,False,True,False,COMPLETE
11,11,0.562,2023-02-13 16:01:55.418044,2023-02-13 16:04:54.906149,0 days 00:02:59.488105,8.459158,0.497546,60,False,True,False,COMPLETE
13,13,0.562,2023-02-13 16:07:34.214612,2023-02-13 16:11:17.505659,0 days 00:03:43.291047,7.394105,0.192465,56,False,True,True,COMPLETE
21,21,0.563,2023-02-13 16:30:13.570936,2023-02-13 16:32:26.975964,0 days 00:02:13.405028,8.862525,0.237644,17,False,True,True,COMPLETE
18,18,0.563,2023-02-13 16:22:29.826937,2023-02-13 16:24:48.352937,0 days 00:02:18.526000,6.502855,0.498010,51,True,False,False,COMPLETE


In [11]:
print(f"Best HistGBoost regression params: {study.best_params}, achieving val RMSE: {study.best_value}")

Best HistGBoost regression params: {'l2_regularization': 8.803675229357003, 'learning_rate': 0.23595804024556916, 'max_leaf_nodes': 20, 'normalization': False, 'scaling': True, 'tfidf': True}, achieving val RMSE: 0.558


In [15]:
save = True
load = False

regressor_params = {"learning_rate": 0.24, "max_leaf_nodes": 20, "l2_regularization": 8.8}
pipe_params = {
    'normalization': False,
    'scaling': True,
    'tfidf': True,
    'requires_dense': True,
}

if load:
    pipe = joblib.load(f"{exp_dir}/hist_gradient_boosting/pipe")
else:
    regressor = HistGradientBoostingRegressor(**regressor_params)
    pipe = sklearn_regressors.train_pipe(regressor, X_train, y_train, **pipe_params)

    if save:
        additional_attributes = {
            "regressor_params": regressor_params,
            "pipe_params": pipe_params,
            "test_programs": test_programs,
        }

        joblib.dump(pipe, f"{exp_dir}/hist_gradient_boosting/pipe")
        with open(f"{exp_dir}/hist_gradient_boosting/additional_attributes.json", "w") as file:
            json.dump(additional_attributes, file, indent=4)

In [16]:
maes = {}
prog_energy = {}

for test_program in test_programs:
    prog_df = test_df[test_df.program_name == test_program]
    prog_bbs = np.array([" ".join(bb) for bb in prog_df.bb.tolist()])
    test_preds = pipe.predict(prog_bbs)
    true_energies = prog_df.energy.values
    
    maes[test_program] = mean_absolute_error(true_energies, test_preds)
    prog_energy[test_program] = {
        "true_energy": sum(true_energies),
        "pred_energy": sum(test_preds),
    }

print("Mean absolute error for test programs")
print(f"{maes}\n")

print("Total energies for test programs")
print(prog_energy)

Mean absolute error for test programs
{'faa': 0.8123990241181436, 'quicksort': 0.2551870738675057, 'declare': 0.4204020368187791}

Total energies for test programs
{'faa': {'true_energy': 1593.1851865544559, 'pred_energy': 596.5419792575909}, 'quicksort': {'true_energy': 4246.078264105905, 'pred_energy': 7377.537371436673}, 'declare': {'true_energy': 6382.836181449226, 'pred_energy': 3948.5523294498194}}
